# EDA of Google Trends data

In [91]:
from linearmodels.panel import PooledOLS, PanelOLS

import statsmodels.api as sm
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf

In [2]:
trend_month_state = pd.read_csv("data/cleaned/trend_month_state.csv", index_col=0)
trend_month_DMA = pd.read_csv("data/cleaned/trend_month_DMA.csv", index_col=0)
trend_all_time_state = pd.read_csv("data/cleaned/trend_all_time_state.csv", index_col=0)
trend_all_time_DMA = pd.read_csv("data/cleaned/trend_all_time_DMA.csv", index_col=0)

In [3]:
trend_all_time_state.replace(0, np.nan, inplace = True)
trend_month_state.replace(0, np.nan, inplace = True)

## EDA

In [4]:
trend_all_time_state.head()

,date_range,state,covid_conspiracy,covid_hoax,pizzagate,plandemic,wuhan_lab
0,2020-03-01 2021-01-31,Alabama,52.0,38.0,80.0,57.0,39.0
1,2020-03-01 2021-01-31,Alaska,90.0,94.0,49.0,77.0,46.0
2,2020-03-01 2021-01-31,Arizona,63.0,48.0,79.0,53.0,49.0
3,2020-03-01 2021-01-31,Arkansas,51.0,46.0,79.0,51.0,52.0
4,2020-03-01 2021-01-31,California,54.0,49.0,56.0,58.0,49.0


In [5]:
trend_all_time_state[
    ["covid_conspiracy", "covid_hoax", "pizzagate", "plandemic", "wuhan_lab"]
].corr(method="pearson")

,covid_conspiracy,covid_hoax,pizzagate,plandemic,wuhan_lab
covid_conspiracy,1.000000,0.253764,-0.120353,0.487821,0.135147
covid_hoax,0.253764,1.000000,-0.112811,0.317223,0.234428
pizzagate,-0.120353,-0.112811,1.000000,-0.239771,-0.300099
plandemic,0.487821,0.317223,-0.239771,1.000000,0.128978
wuhan_lab,0.135147,0.234428,-0.300099,0.128978,1.000000


It turns out that popularity of COVID-related conspiracy theories is not strongly correlated with "pizzagate", which is not related to COVID-19 per se. Among all covid-19 related keywords, there is a positive, yet not necessarily strong correlation. 

### Principle component analyses

In [6]:
pca_pipe =  Pipeline([('inputer', SimpleImputer()), ('PCA', PCA(n_components=2))])
principalComponents = pca_pipe.fit_transform(
    trend_all_time_state[["covid_conspiracy", "covid_hoax", "plandemic", "wuhan_lab"]]
)
principalDf = pd.DataFrame(data=principalComponents)

In [7]:
pca_pipe["PCA"].explained_variance_ratio_

array([0.46055991, 0.23254269])

In [8]:
pca_pipe["PCA"].components_

array([[ 0.62681881,  0.47529325,  0.54963197,  0.28124579],
       [-0.43683202,  0.43500537, -0.25854643,  0.74370818]])

## Panel analyses

## Develop an index

### with "Wuhan Lab"

In [9]:
trend_all_time_state["index_1"] = trend_all_time_state[["covid_hoax", "covid_conspiracy", "plandemic", "wuhan_lab"]].mean(axis = 1)

### without "Wuhan Lab"

In [10]:
trend_all_time_state["index_2"] = trend_all_time_state[["covid_hoax", "covid_conspiracy", "plandemic"]].mean(axis = 1)

In [11]:
trend_all_time_state.sort_values("index_2", ascending = False)

,date_range,state,covid_conspiracy,covid_hoax,pizzagate,plandemic,wuhan_lab,index_1,index_2
1,2020-03-01 2021-01-31,Alaska,90.0,94.0,49.0,77.0,46.0,76.750000,87.000000
29,2020-03-01 2021-01-31,New Hampshire,92.0,78.0,55.0,65.0,91.0,81.500000,78.333333
45,2020-03-01 2021-01-31,Vermont,55.0,100.0,48.0,79.0,100.0,83.500000,78.000000
23,2020-03-01 2021-01-31,Minnesota,82.0,50.0,57.0,94.0,52.0,69.500000,75.333333
34,2020-03-01 2021-01-31,North Dakota,100.0,39.0,54.0,85.0,38.0,65.500000,74.666667
27,2020-03-01 2021-01-31,Nebraska,78.0,58.0,72.0,81.0,39.0,64.000000,72.333333
50,2020-03-01 2021-01-31,Wyoming,71.0,NaN,61.0,72.0,32.0,58.333333,71.500000
37,2020-03-01 2021-01-31,Oregon,70.0,65.0,67.0,75.0,43.0,63.250000,70.000000
5,2020-03-01 2021-01-31,Colorado,77.0,67.0,63.0,66.0,35.0,61.250000,70.000000
12,2020-03-01 2021-01-31,Idaho,53.0,89.0,69.0,66.0,42.0,62.500000,69.333333


## Regression analyses

### Relationship between conspiracy theory popularity and compliance (mobility)

In [12]:
us_daily = pd.read_csv("data/cleaned/daily_df.csv", index_col=0)
us_daily["date"] = pd.to_datetime(us_daily["date"])
us_daily = us_daily.set_index("date")

In [13]:
us_monthly = us_daily.groupby([pd.Grouper(freq='M'), 'state']).mean()
us_monthly

positive  probableCases      negative  \
date       state                                                       
2020-01-31 Florida             0.000000            NaN           NaN   
           Massachusetts            NaN            NaN           NaN   
           Virginia                 NaN            NaN           NaN   
           Washington          1.333333            NaN           NaN   
2020-02-29 Florida             0.000000            NaN           NaN   
...                                 ...            ...           ...   
2021-02-28 Virginia       533521.941176  108944.411765           NaN   
           Washington     321918.647059   15809.117647           NaN   
           West Virginia  125328.235294   24920.941176  1.907528e+06   
           Wisconsin      602149.941176   51173.529412  2.543102e+06   
           Wyoming         52792.588235    7784.411765  1.741915e+05   

                             pending  totalTestResults  hospitalizedCurrently  \
date       state                                                                
2020-01-31 Florida               NaN      2.000000e+00                    NaN   
           Massachusetts         NaN      2.700000e+00                    NaN   
           Virginia              NaN               NaN                    NaN   
           Washington            NaN      0.000000e+00                    NaN   
2020-02-29 Florida               NaN      1.024138e+01                    NaN   
...                              ...               ...                    ...   
2021-02-28 Virginia       251.411765      5.468171e+06            2196.411765   
           Washington            NaN      4.761709e+06             740.529412   
           West Virginia         NaN      2.032857e+06             373.470588   
           Wisconsin       38.176471      6.377024e+06             531.882353   
           Wyoming               NaN      6.367162e+05              43.176471   

                          hospitalizedCumulative  inIcuCurrently  \
date       state                                                   
2020-01-31 Florida                           NaN             NaN   
           Massachusetts                     NaN             NaN   
           Virginia                          NaN             NaN   
           Washington                        NaN             NaN   
2020-02-29 Florida                           NaN             NaN   
...                                          ...             ...   
2021-02-28 Virginia                 22329.588235      446.235294   
           Washington               18326.117647      175.411765   
           West Virginia                     NaN      100.058824   
           Wisconsin                24978.058824      137.176471   
           Wyoming                   1315.470588             NaN   

                          inIcuCumulative  onVentilatorCurrently  ...  \
date       state                                                  ...   
2020-01-31 Florida                    NaN                    NaN  ...   
           Massachusetts              NaN                    NaN  ...   
           Virginia                   NaN                    NaN  ...   
           Washington                 NaN                    NaN  ...   
2020-02-29 Florida                    NaN                    NaN  ...   
...                                   ...                    ...  ...   
2021-02-28 Virginia                   NaN             280.117647  ...   
           Washington                 NaN              69.176471  ...   
           West Virginia              NaN              47.000000  ...   
           Wisconsin          2206.823529                    NaN  ...   
           Wyoming                    NaN                    NaN  ...   

                          C6_Stay at home requirements  C6_Flag  \
date       state                                                  
2020-01-31 Florida                                 0.0      NaN   
    

In [14]:
us_monthly = pd.DataFrame(us_monthly.to_records())

In [15]:
trend_month_state["date"] = pd.to_datetime(trend_month_state["date_range"].str.slice(start = -10))

In [16]:
trend_month_state = trend_month_state.drop("date_range", axis = 1)

In [17]:
trend_month_state

,state,covid_conspiracy,covid_hoax,pizzagate,plandemic,wuhan_lab,date
0,Alabama,29.0,10.0,76.0,NaN,33.0,2020-03-31
1,Alaska,73.0,80.0,97.0,NaN,100.0,2020-03-31
2,Arizona,34.0,25.0,60.0,NaN,31.0,2020-03-31
3,Arkansas,100.0,NaN,11.0,NaN,34.0,2020-03-31
4,California,41.0,18.0,44.0,18.0,42.0,2020-03-31
...,...,...,...,...,...,...,...
556,Virginia,19.0,14.0,10.0,5.0,14.0,2021-01-31
557,Washington,23.0,4.0,12.0,NaN,52.0,2021-01-31
558,West Virginia,NaN,NaN,35.0,NaN,NaN,2021-01-31
559,Wisconsin,11.0,6.0,45.0,8.0,NaN,2021-01-31


In [18]:
us_monthly = pd.merge(trend_month_state, us_monthly, on = ["state", "date"], how = "left")

In [19]:
us_monthly = us_monthly.set_index("date")

#### Cross-sectional

In [20]:
us_cross_sectional = us_daily.groupby("state").mean()

In [21]:
us_cross_sectional.head()

,positive,probableCases,negative,pending,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,C6_Stay at home requirements,C6_Flag,C7_Restrictions on internal movement,C7_Flag,C8_International travel controls,ConfirmedCases,ConfirmedDeaths,StringencyIndex,GovernmentResponseIndex,ContainmentHealthIndex
state,,,,,,,,,,,,,,,,,,,,,
Alabama,152878.247126,32034.665399,8.128135e+05,33.000000,9.344232e+05,1184.939577,16614.212500,NaN,1531.476038,NaN,...,1.057803,0.966565,0.000000,NaN,0.000000,153084.663793,2474.939655,35.718439,40.565723,42.816040
Alaska,15237.020710,NaN,NaN,8.500000,5.100322e+05,56.078431,389.485549,NaN,NaN,7.664151,...,0.758621,0.116883,1.706897,0.92638,1.936782,15512.859599,75.020057,53.132787,52.471810,53.053736
American Samoa,0.000000,NaN,1.253120e+03,10.500000,1.197673e+03,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arizona,230197.700855,12375.980159,1.094707e+06,44.240000,2.228371e+06,1982.344051,18215.883191,531.459807,NaN,347.585209,...,0.643678,1.000000,0.212644,1.00000,0.212644,230206.666667,4653.578348,42.127989,47.673678,46.597615
Arkansas,91312.756447,24916.214286,8.395289e+05,47.588235,9.188568e+05,538.387387,5427.229102,308.421384,43.000000,100.003030,...,0.282421,1.000000,0.236311,1.00000,0.236311,91338.467049,1449.977077,44.070519,45.216657,47.682075


In [22]:
trend_all_time_state.drop("date_range", axis = 1)

,state,covid_conspiracy,covid_hoax,pizzagate,plandemic,wuhan_lab,index_1,index_2
0,Alabama,52.0,38.0,80.0,57.0,39.0,46.500000,49.000000
1,Alaska,90.0,94.0,49.0,77.0,46.0,76.750000,87.000000
2,Arizona,63.0,48.0,79.0,53.0,49.0,53.250000,54.666667
3,Arkansas,51.0,46.0,79.0,51.0,52.0,50.000000,49.333333
4,California,54.0,49.0,56.0,58.0,49.0,52.500000,53.666667
5,Colorado,77.0,67.0,63.0,66.0,35.0,61.250000,70.000000
6,Connecticut,39.0,49.0,48.0,63.0,64.0,53.750000,50.333333
7,Delaware,17.0,60.0,78.0,28.0,60.0,41.250000,35.000000
8,District of Columbia,60.0,45.0,28.0,85.0,79.0,67.250000,63.333333
9,Florida,50.0,39.0,59.0,45.0,44.0,44.500000,44.666667


In [23]:
us_cross_sectional = pd.merge(us_cross_sectional, trend_all_time_state.drop("date_range", axis = 1), on="state")

In [24]:
us_cross_sectional[
    [
        "covid_conspiracy",
        "covid_hoax",
        "pizzagate",
        "plandemic",
        "wuhan_lab",
        "index_1",
        "index_2",
        "retail_and_recreation_percent_change_from_baseline",
        "grocery_and_pharmacy_percent_change_from_baseline",
        "parks_percent_change_from_baseline",
        "transit_stations_percent_change_from_baseline",
        "workplaces_percent_change_from_baseline",
        "residential_percent_change_from_baseline",
    ]
].corr()

,covid_conspiracy,covid_hoax,pizzagate,plandemic,wuhan_lab,index_1,index_2,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
covid_conspiracy,1.000000,0.253764,-0.120353,0.487821,0.135147,0.721600,0.784104,-0.012550,-0.022287,0.179848,-0.053732,0.069281,-0.004129
covid_hoax,0.253764,1.000000,-0.112811,0.317223,0.234428,0.673981,0.691606,0.222925,0.241081,0.298880,0.119585,0.125759,-0.126679
pizzagate,-0.120353,-0.112811,1.000000,-0.239771,-0.300099,-0.286249,-0.208420,0.404760,0.307774,-0.000874,0.404453,0.366606,-0.485480
plandemic,0.487821,0.317223,-0.239771,1.000000,0.128978,0.724384,0.787842,0.011192,0.214272,0.443996,0.063424,0.057990,-0.038358
wuhan_lab,0.135147,0.234428,-0.300099,0.128978,1.000000,0.544144,0.205043,-0.329921,-0.395268,0.020604,-0.454625,-0.403559,0.430357
index_1,0.721600,0.673981,-0.286249,0.724384,0.544144,1.000000,0.931693,-0.033854,0.019215,0.354191,-0.113211,-0.046036,0.088378
index_2,0.784104,0.691606,-0.208420,0.787842,0.205043,0.931693,1.000000,0.107221,0.199177,0.408520,0.073439,0.126204,-0.088919
retail_and_recreation_percent_change_from_baseline,-0.012550,0.222925,0.404760,0.011192,-0.329921,-0.033854,0.107221,1.000000,0.874400,0.622192,0.859456,0.919312,-0.915298
grocery_and_pharmacy_percent_change_from_baseline,-0.022287,0.241081,0.307774,0.214272,-0.395268,0.019215,0.199177,0.874400,1.000000,0.686546,0.845084,0.826631,-0.835964
parks_percent_change_from_baseline,0.179848,0.298880,-0.000874,0.443996,0.020604,0.354191,0.408520,0.622192,0.686546,1.000000,0.489512,0.537091,-0.473140


In [25]:
us_cross_sectional.columns

Index(['state', 'positive', 'probableCases', 'negative', 'pending',
       'totalTestResults', 'hospitalizedCurrently', 'hospitalizedCumulative',
       'inIcuCurrently', 'inIcuCumulative', 'onVentilatorCurrently',
       'onVentilatorCumulative', 'recovered', 'totalTestsViral',
       'positiveTestsViral', 'negativeTestsViral', 'positiveCasesViral',
       'deathConfirmed', 'deathProbable', 'totalTestEncountersViral',
       'totalTestsPeopleViral', 'totalTestsAntibody', 'positiveTestsAntibody',
       'negativeTestsAntibody', 'totalTestsPeopleAntibody',
       'positiveTestsPeopleAntibody', 'negativeTestsPeopleAntibody',
       'totalTestsPeopleAntigen', 'positiveTestsPeopleAntigen',
       'totalTestsAntigen', 'positiveTestsAntigen', 'fips', 'positiveIncrease',
       'negativeIncrease', 'total', 'totalTestResultsIncrease', 'posNeg',
       'dataQualityGrade', 'deathIncrease', 'hospitalizedIncrease',
       'commercialScore', 'negativeRegularScore', 'negativeScore',
       'positive

In [26]:
results = smf.ols('non_residential_percent_change_from_baseline ~ index_2 + ContainmentHealthIndex', data=us_cross_sectional).fit()

In [27]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                         OLS Regression Results                                         
========================================================================================================
Dep. Variable:     non_residential_percent_change_from_baseline   R-squared:                       0.318
Model:                                                      OLS   Adj. R-squared:                  0.289
Method:                                           Least Squares   F-statistic:                     10.97
Date:                                          Fri, 19 Feb 2021   Prob (F-statistic):           0.000123
Time:                                                  17:09:40   Log-Likelihood:                -185.51
No. Observations:                                            50   AIC:                             377.0
Df Residuals:                                                47   BIC:                             382.8
Df Model:                                                     2                                         
Covariance Type:                                      nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  5.1563     11.068      0.466      0.643     -17.110      27.423
index_2                    0.3525      0.119      2.960      0.005       0.113       0.592
ContainmentHealthIndex    -0.6122      0.170     -3.592      0.001      -0.955      -0.269
==============================================================================
Omnibus:                        3.329   Durbin-Watson:                   2.137
Prob(Omnibus):                  0.189   Jarque-Bera (JB):                2.688
Skew:                          -0.566   Prob(JB):                        0.261
Kurtosis:                       3.086   Cond. No.                         589.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
results = smf.ols('non_residential_percent_change_from_baseline ~ plandemic + ContainmentHealthIndex', data=us_cross_sectional).fit()

In [29]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                         OLS Regression Results                                         
========================================================================================================
Dep. Variable:     non_residential_percent_change_from_baseline   R-squared:                       0.347
Model:                                                      OLS   Adj. R-squared:                  0.320
Method:                                           Least Squares   F-statistic:                     12.51
Date:                                          Fri, 19 Feb 2021   Prob (F-statistic):           4.40e-05
Time:                                                  17:09:40   Log-Likelihood:                -184.42
No. Observations:                                            50   AIC:                             374.8
Df Residuals:                                                47   BIC:                             380.6
Df Model:                                                     2                                         
Covariance Type:                                      nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  2.3147     10.942      0.212      0.833     -19.698      24.327
plandemic                  0.3187      0.095      3.356      0.002       0.128       0.510
ContainmentHealthIndex    -0.5506      0.168     -3.278      0.002      -0.889      -0.213
==============================================================================
Omnibus:                        2.077   Durbin-Watson:                   2.321
Prob(Omnibus):                  0.354   Jarque-Bera (JB):                1.825
Skew:                          -0.461   Prob(JB):                        0.402
Kurtosis:                       2.837   Cond. No.                         629.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
results = smf.ols('non_residential_percent_change_from_baseline ~ covid_hoax + ContainmentHealthIndex', data=us_cross_sectional).fit()

In [31]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                         OLS Regression Results                                         
========================================================================================================
Dep. Variable:     non_residential_percent_change_from_baseline   R-squared:                       0.303
Model:                                                      OLS   Adj. R-squared:                  0.273
Method:                                           Least Squares   F-statistic:                     10.01
Date:                                          Fri, 19 Feb 2021   Prob (F-statistic):           0.000246
Time:                                                  17:09:40   Log-Likelihood:                -181.38
No. Observations:                                            49   AIC:                             368.8
Df Residuals:                                                46   BIC:                             374.4
Df Model:                                                     2                                         
Covariance Type:                                      nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 15.4988      9.244      1.677      0.100      -3.109      34.106
covid_hoax                 0.2459      0.092      2.671      0.010       0.061       0.431
ContainmentHealthIndex    -0.6809      0.171     -3.972      0.000      -1.026      -0.336
==============================================================================
Omnibus:                        2.637   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.268   Jarque-Bera (JB):                1.656
Skew:                          -0.347   Prob(JB):                        0.437
Kurtosis:                       3.574   Cond. No.                         472.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

It looks like popularity of terms like "plandemic" is a good predictor of high mobility, after controlling for containment index.

### Panel regressions: on mobility

#### Pooled OLS

In [ ]:
df_onegrp.set_index(["date"]).shift(1)

In [32]:
us_monthly.columns

Index(['state', 'covid_conspiracy', 'covid_hoax', 'pizzagate', 'plandemic',
       'wuhan_lab', 'positive', 'probableCases', 'negative', 'pending',
       'totalTestResults', 'hospitalizedCurrently', 'hospitalizedCumulative',
       'inIcuCurrently', 'inIcuCumulative', 'onVentilatorCurrently',
       'onVentilatorCumulative', 'recovered', 'totalTestsViral',
       'positiveTestsViral', 'negativeTestsViral', 'positiveCasesViral',
       'deathConfirmed', 'deathProbable', 'totalTestEncountersViral',
       'totalTestsPeopleViral', 'totalTestsAntibody', 'positiveTestsAntibody',
       'negativeTestsAntibody', 'totalTestsPeopleAntibody',
       'positiveTestsPeopleAntibody', 'negativeTestsPeopleAntibody',
       'totalTestsPeopleAntigen', 'positiveTestsPeopleAntigen',
       'totalTestsAntigen', 'positiveTestsAntigen', 'fips', 'positiveIncrease',
       'negativeIncrease', 'total', 'totalTestResultsIncrease', 'posNeg',
       'dataQualityGrade', 'deathIncrease', 'hospitalizedIncrease',
   

In [33]:
us_monthly[["non_residential_percent_change_from_baseline"]].shift(1)

,non_residential_percent_change_from_baseline
date,
2020-03-31,NaN
2020-03-31,-1.880000
2020-03-31,-7.107692
2020-03-31,-10.207143
2020-03-31,1.300000
...,...
2021-01-31,-26.858065
2021-01-31,-24.741935
2021-01-31,-26.290323


In [69]:
val = (
    us_monthly[["state", "covid_conspiracy", "covid_hoax", "pizzagate", "plandemic", "wuhan_lab"]]
    .groupby("state")
    .resample("M")
    .asfreq()
).groupby(level=0).shift(1) 

In [68]:
pd.DataFrame(val.to_records(index=False))

,state,covid_conspiracy,covid_hoax,pizzagate,plandemic,wuhan_lab
0,NaN,NaN,NaN,NaN,NaN,NaN
1,Alabama,29.0,10.0,76.0,NaN,33.0
2,Alabama,44.0,37.0,48.0,NaN,54.0
3,Alabama,33.0,14.0,15.0,45.0,44.0
4,Alabama,NaN,51.0,71.0,57.0,29.0
...,...,...,...,...,...,...
556,Wyoming,NaN,NaN,50.0,NaN,NaN
557,Wyoming,NaN,NaN,77.0,NaN,NaN
558,Wyoming,NaN,NaN,100.0,NaN,NaN
559,Wyoming,95.0,NaN,NaN,NaN,NaN


In [83]:
us_monthly = us_monthly.reset_index().set_index(["state", "date"])

In [110]:
us_monthly["StringencyIndex_times_plandemic"] = us_monthly["StringencyIndex"] * us_monthly["plandemic"] 

In [104]:
exog_vars = ["plandemic", "ContainmentHealthIndex", "StringencyIndex"]
exog = sm.add_constant(us_monthly[exog_vars])
mod = PooledOLS(us_monthly.non_residential_percent_change_from_baseline, exog)
pooled_res = mod.fit()
print(pooled_res)

                                       PooledOLS Estimation Summary                                       
Dep. Variable:     non_residential_percent_change_from_baseline   R-squared:                        0.2078
Estimator:                                            PooledOLS   R-squared (Between):              0.3019
No. Observations:                                           332   R-squared (Within):               0.0623
Date:                                          Fri, Feb 19 2021   R-squared (Overall):              0.2078
Time:                                                  18:26:30   Log-likelihood                   -1425.0
Cov. Estimator:                                      Unadjusted                                           
                                                                  F-statistic:                      28.680
Entities:                                                    50   P-value                           0.0000
Avg Obs:                             

In [105]:
exog_vars = ["plandemic", "StringencyIndex"]
exog = sm.add_constant(us_monthly[exog_vars])
mod = PooledOLS(us_monthly.non_residential_percent_change_from_baseline, exog)
pooled_res = mod.fit()
print(pooled_res)

                                       PooledOLS Estimation Summary                                       
Dep. Variable:     non_residential_percent_change_from_baseline   R-squared:                        0.2065
Estimator:                                            PooledOLS   R-squared (Between):              0.3083
No. Observations:                                           332   R-squared (Within):               0.0506
Date:                                          Fri, Feb 19 2021   R-squared (Overall):              0.2065
Time:                                                  18:26:37   Log-likelihood                   -1425.2
Cov. Estimator:                                      Unadjusted                                           
                                                                  F-statistic:                      42.817
Entities:                                                    50   P-value                           0.0000
Avg Obs:                             

In [111]:
exog_vars = ["plandemic", "StringencyIndex", "StringencyIndex_times_plandemic"]
exog = sm.add_constant(us_monthly[exog_vars])
mod = PooledOLS(us_monthly.non_residential_percent_change_from_baseline, exog)
pooled_res = mod.fit()
print(pooled_res)

                                       PooledOLS Estimation Summary                                       
Dep. Variable:     non_residential_percent_change_from_baseline   R-squared:                        0.2085
Estimator:                                            PooledOLS   R-squared (Between):              0.3117
No. Observations:                                           332   R-squared (Within):               0.0539
Date:                                          Fri, Feb 19 2021   R-squared (Overall):              0.2085
Time:                                                  18:31:01   Log-likelihood                   -1424.8
Cov. Estimator:                                      Unadjusted                                           
                                                                  F-statistic:                      28.805
Entities:                                                    50   P-value                           0.0000
Avg Obs:                             

#### Fixed effects

In [107]:
exog_vars = ["plandemic", "StringencyIndex"]
exog = sm.add_constant(us_monthly[exog_vars])
mod = PanelOLS(us_monthly.non_residential_percent_change_from_baseline, exog, entity_effects=True)
pooled_res = mod.fit()
print(pooled_res)

                                       PanelOLS Estimation Summary                                        
Dep. Variable:     non_residential_percent_change_from_baseline   R-squared:                        0.0778
Estimator:                                             PanelOLS   R-squared (Between):              0.2378
No. Observations:                                           332   R-squared (Within):               0.0778
Date:                                          Fri, Feb 19 2021   R-squared (Overall):              0.1785
Time:                                                  18:26:46   Log-likelihood                   -1312.7
Cov. Estimator:                                      Unadjusted                                           
                                                                  F-statistic:                      11.809
Entities:                                                    50   P-value                           0.0000
Avg Obs:                             

In [114]:
exog_vars = ["plandemic", "StringencyIndex", "StringencyIndex_times_plandemic"]
exog = sm.add_constant(us_monthly[exog_vars])
mod = PanelOLS(us_monthly.non_residential_percent_change_from_baseline, exog, entity_effects=True)
pooled_res = mod.fit()
print(pooled_res)

                                       PanelOLS Estimation Summary                                        
Dep. Variable:     non_residential_percent_change_from_baseline   R-squared:                        0.0817
Estimator:                                             PanelOLS   R-squared (Between):              0.2427
No. Observations:                                           332   R-squared (Within):               0.0817
Date:                                          Fri, Feb 19 2021   R-squared (Overall):              0.1799
Time:                                                  18:33:00   Log-likelihood                   -1312.0
Cov. Estimator:                                      Unadjusted                                           
                                                                  F-statistic:                      8.2691
Entities:                                                    50   P-value                           0.0000
Avg Obs:                             

In [121]:
exog_vars = ["plandemic", "StringencyIndex", "StringencyIndex_times_plandemic"]
exog = sm.add_constant(us_monthly[exog_vars])
mod = PanelOLS(us_monthly.non_residential_percent_change_from_baseline, exog, entity_effects=True, time_effects=True)
pooled_res = mod.fit()
print(pooled_res)

                                       PanelOLS Estimation Summary                                        
Dep. Variable:     non_residential_percent_change_from_baseline   R-squared:                        0.0695
Estimator:                                             PanelOLS   R-squared (Between):              0.1268
No. Observations:                                           332   R-squared (Within):               0.0771
Date:                                          Fri, Feb 19 2021   R-squared (Overall):              0.1153
Time:                                                  19:13:51   Log-likelihood                   -1099.4
Cov. Estimator:                                      Unadjusted                                           
                                                                  F-statistic:                      5.0028
Entities:                                                    50   P-value                           0.0007
Avg Obs:                             

### Panel regressions: on mobility

In [122]:
#### Pooled OLS

In [125]:
us_cross_sectional["positivityRate"] = us_cross_sectional["positiveIncrease"] / us_cross_sectional["totalTestResultsIncrease"]

In [126]:
us_cross_sectional.columns

Index(['state', 'positive', 'probableCases', 'negative', 'pending',
       'totalTestResults', 'hospitalizedCurrently', 'hospitalizedCumulative',
       'inIcuCurrently', 'inIcuCumulative', 'onVentilatorCurrently',
       'onVentilatorCumulative', 'recovered', 'totalTestsViral',
       'positiveTestsViral', 'negativeTestsViral', 'positiveCasesViral',
       'deathConfirmed', 'deathProbable', 'totalTestEncountersViral',
       'totalTestsPeopleViral', 'totalTestsAntibody', 'positiveTestsAntibody',
       'negativeTestsAntibody', 'totalTestsPeopleAntibody',
       'positiveTestsPeopleAntibody', 'negativeTestsPeopleAntibody',
       'totalTestsPeopleAntigen', 'positiveTestsPeopleAntigen',
       'totalTestsAntigen', 'positiveTestsAntigen', 'fips', 'positiveIncrease',
       'negativeIncrease', 'total', 'totalTestResultsIncrease', 'posNeg',
       'dataQualityGrade', 'deathIncrease', 'hospitalizedIncrease',
       'commercialScore', 'negativeRegularScore', 'negativeScore',
       'positive

In [131]:
results = smf.ols('positivityRate ~ plandemic + ContainmentHealthIndex', data=us_cross_sectional).fit()

In [132]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         positivityRate   R-squared:                       0.486
Model:                            OLS   Adj. R-squared:                  0.464
Method:                 Least Squares   F-statistic:                     22.19
Date:                Fri, 19 Feb 2021   Prob (F-statistic):           1.64e-07
Time:                        19:23:46   Log-Likelihood:                 88.182
No. Observations:                  50   AIC:                            -170.4
Df Residuals:                      47   BIC:                            -164.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                  0.3729      0.047      7.947      0.000       0.278       0.467
plandemic                 -0.0006      0.000     -1.496      0.141      -0.001       0.000
ContainmentHealthIndex    -0.0048      0.001     -6.625      0.000      -0.006      -0.003
==============================================================================
Omnibus:                       16.844   Durbin-Watson:                   1.843
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               20.713
Skew:                           1.244   Prob(JB):                     3.18e-05
Kurtosis:                       4.937   Cond. No.                         629.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [137]:
us_cross_sectional[["state", "positivityRate"]].sort_values("positivityRate")


,state,positivityRate
45,Vermont,0.014178
11,Hawaii,0.026546
19,Maine,0.027931
8,District of Columbia,0.033275
1,Alaska,0.033995
21,Massachusetts,0.037114
6,Connecticut,0.042621
39,Rhode Island,0.043567
32,New York,0.043604
37,Oregon,0.044116
